In [1]:
!pip install bitsandbytes -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 35.6 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.1 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.5 MB/s eta 0:00:0000:0100:01m


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

VER=1
# model_name = "unsloth/gemma-2-9b-it"
model_name = '/kaggle/input/vseros-rustore-gemma-2-9b-it-full-data/ver_1'
# model_name = "/kaggle/input/qwen2.5/transformers/7b/1"
EPOCHS = 2

DIR = f"ver_{VER}"
os.makedirs(DIR, exist_ok=True)

In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/vseros-rustore-1stage/train.tsv', delimiter='\t')
test = pd.read_csv('/kaggle/input/vseros-rustore-1stage/test.tsv', delimiter='\t')
sub = pd.read_csv('/kaggle/input/vseros-rustore-1stage/sample_submission.tsv', delimiter='\t')

In [4]:
def build_text(row):
    parts = [
        str(row.get("app_name") or ""),
        str(row.get("shortDescription") or ""),
        str(row.get("full_description") or ""),
    ]
    txt = " ".join(p.strip() for p in parts if p).strip()
    return " ".join(txt.split())  # нормализуем пробелы

train["text"] = train.apply(build_text, axis=1)

In [5]:
expected_cols = {"app_name", "full_description", "shortDescription", "labels_str"}

def parse_labels(s):

    """Преобразует строку вида 'cat1|cat2' в список ['cat1', 'cat2']"""

    if pd.isna(s):
       return []
    return [t.strip() for t in str(s).split("|") if t.strip()]

train["labels_list"] = train["labels_str"].apply(parse_labels)

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
import json
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(train["labels_list"])
labels = mlb.classes_.tolist()
NUM_LABELS = len(labels)

with open("labels.json", "w", encoding="utf-8") as f:
    json.dump(labels, f, ensure_ascii=False, indent=2)

print("Число меток:", NUM_LABELS)
print("Пример меток:", labels[:10])

Число меток: 45
Пример меток: ['action', 'adsAndServices', 'adventure', 'ar', 'arcade', 'board', 'boardAndCard', 'books', 'business', 'card']


In [7]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
MODEL_ID = model_name
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_texts(texts):
    return tokenizer(texts, truncation=True, padding=False, max_length=MAX_LEN)

# простой сплит
tr_texts, val_texts, tr_y, val_y = train_test_split(
    train["text"].tolist(), Y, test_size=0.15, random_state=42
)

tr_enc = tokenize_texts(tr_texts)
val_enc = tokenize_texts(val_texts)

In [8]:
import torch
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer)
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.enc = encodings
        self.labels = labels
    def __len__(self):
        return len(self.enc["input_ids"])
    def __getitem__(self, i):
        item = {k: torch.tensor(v[i]) for k, v in self.enc.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[i]).float()
        return item

train_ds = Dataset(tr_enc, tr_y)
val_ds = Dataset(val_enc, val_y)
collator = DataCollatorWithPadding(tokenizer)

2025-10-04 17:02:26.051239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759597346.229885     100 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759597346.281943     100 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )


model = AutoModelForSequenceClassification.from_pretrained(
    "unsloth/gemma-2-9b-it",
    num_labels=NUM_LABELS,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    # quantization_config=bnb_config,
    trust_remote_code=True,
    problem_type="multi_label_classification"
)

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, model_name)

In [11]:
def hitrate_at_k(logits, y_true, k = 3):
    topk = np.argpartition(-logits, kth=k-1, axis=1)[:, :k]
    hits = 0
    for i in range(len(y_true)):
        true_idx = np.where(y_true[i] > 0)[0]
        if len(set(true_idx) & set(topk[i])) > 0:
            hits += 1
    return hits / len(y_true)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return {"H@3": hitrate_at_k(logits, labels, k=3)}

In [12]:
# from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=8,  # Rank
#     lora_alpha=8,
#     target_modules=["q_proj", "k_proj", "v_proj",
#                     "down_proj","up_proj","o_proj","gate_proj"],  # модули для адаптации
#     lora_dropout=0.05,
#     bias="none",
#     task_type="SEQ_CLS"
# )

# model = get_peft_model(model, lora_config)

In [13]:
training_args = TrainingArguments(
    output_dir = f"./{DIR}",
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch", #no for no saving 
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    logging_dir="./logs",
    logging_steps=250,
    save_steps=1,
    eval_steps=1,
    optim='adamw_8bit',
    lr_scheduler_type="linear",
    save_total_limit=1,
    metric_for_best_model="H@3",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="none",
    bf16=True, # TRAIN WITH BF16 IF LOCAL GPU IS NEWER GPU          
    fp16=False, # INFER WITH FP16 BECAUSE KAGGLE IS T4 GPU
)

In [14]:
model.config.pad_token_id = model.config.eos_token_id

In [15]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     # eval_dataset=val_ds,
#     tokenizer=tokenizer,
#     compute_metrics=compute_map3,
# )

# trainer.train()

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    data_collator=collator, # Инструмент для создания батчей (с паддингом)
    compute_metrics=compute_metrics
)
# trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
# trainer.save_model(f"ver_{VER}")      
# tokenizer.save_pretrained(f"ver_{VER}")

In [18]:
test["text"] = test.apply(build_text, axis=1)

# токенизируем
te_enc = tokenize_texts(test["text"].tolist())
test_ds = Dataset(te_enc, labels=None)

# предсказания (логиты)
pred_logits = trainer.predict(test_ds).predictions

# берём top-3 меток и сортируем их по убыванию уверенности
id2label = np.array(labels)
top3_idx = np.argpartition(-pred_logits, kth=2, axis=1)[:, :3]

# упорядочим эти три по логитам
ordered = np.take_along_axis(
    top3_idx,
    np.argsort(-np.take_along_axis(pred_logits, top3_idx, axis=1), axis=1),
    axis=1
)

# gреобразуем отсортированные индексы в строки с метками, разделенными '|'
pred_top3 = ["|".join(id2label[row]) for row in ordered]

# DataFrame для сабмита
submission = pd.DataFrame({
    "app_name": test["app_name"],
    "labels_str": pred_top3
})
submission.to_csv("submission_gemma9b-fd.tsv", sep="\t", index=False)
submission.head()

app_name                 labels_str
0             Lemon clicker        casual|arcade|indie
1      Memo Английский язык  education|tools|parenting
2          Slave Man Rescue    adventure|puzzle|arcade
3     Taking Care of Granny  casual|children|adventure
4  Escape From Classic Room       puzzle|casual|arcade

In [19]:
pred_probs = torch.sigmoid(torch.tensor(pred_logits)).numpy()

In [20]:
np.save('gemma-9b-fd_logits.npy', pred_logits)
np.save('gemma-9b-fd_probs.npy', pred_probs)